In [73]:
%matplotlib inline

# SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead
# This is a warning and not an error
import warnings
warnings.filterwarnings('ignore')

# Import libraries
import pandas as pd
import matplotlib as plt

# This was used for our t-test
import scipy
from scipy.stats import ttest_ind
#import scipy.stats as stats

# This was to change our date type from int64 to datetime64
import datetime
# Output File (CSV), need 3 files for 3 sets of data for comparison
output_data_file_1 = "output_data/data_source_1.csv"
output_data_file_2 = "output_data/data_source_2.csv"
output_data_file_3 = "output_data/data_source_3.csv"

Covid-19 Data Cleanup
Three resources were used for data analysis.

The null hypothesis is that our populations are statistically significantly different

In [74]:
#Load data for source 1

#Resource = https://covidtracking.com/data/download

# Load file 1 to read
covid_data_1 = "all-states-history.csv"

# Read csv file into DataFrame, set state as index
covid_df_1 = pd.read_csv(covid_data_1, index_col = "state")
covid_df_1

,date,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
state,,,,,,,,,,,,,,,,,,,,,
AK,3/7/21,305.0,NaN,0,NaN,1293.0,1293.0,33.0,0,NaN,...,1731628.0,0,NaN,NaN,NaN,NaN,NaN,0,1731628.0,0
AL,3/7/21,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,2676.0,...,2323788.0,2347,NaN,NaN,119757.0,NaN,2323788.0,2347,NaN,0
AR,3/7/21,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,NaN,...,2736442.0,3380,NaN,NaN,NaN,481311.0,NaN,0,2736442.0,3380
AS,3/7/21,0.0,NaN,0,NaN,NaN,NaN,NaN,0,NaN,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
AZ,3/7/21,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,NaN,...,7908105.0,45110,580569.0,NaN,444089.0,NaN,3842945.0,14856,7908105.0,45110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WA,1/17/20,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
WA,1/16/20,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
WA,1/15/20,NaN,NaN,0,NaN,NaN,NaN,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [75]:
# Narrow down columns to those of interest
df_1 = covid_df_1[["date", "positive", 'negative']]
df_1

,date,positive,negative
state,,,
AK,3/7/21,56886,0
AL,3/7/21,499819,1931711
AR,3/7/21,324818,2480716
AS,3/7/21,0,2140
AZ,3/7/21,826454,3073010
...,...,...,...
WA,1/17/20,0,0
WA,1/16/20,0,0
WA,1/15/20,0,0


In [76]:
# Filter out specific states for analysis - DC, MD, VA
states = ["DC", "MD", "VA"]
filter_df_1  = df_1[df_1.index.isin(states)]
filter_df_1

,date,positive,negative
state,,,
DC,3/7/21,41419,0
MD,3/7/21,387319,3034546
VA,3/7/21,585700,0
DC,3/6/21,41273,0
MD,3/6/21,386610,3032496
...,...,...,...
VA,1/31/20,0,0
VA,1/30/20,0,0
VA,1/29/20,0,0


In [77]:
# ename state
state_name_df = filter_df_1.rename(index={"DC": "Washington DC", "MD": "Maryland", "VA": "Virginia"})
state_name_df

,date,positive,negative
state,,,
Washington DC,3/7/21,41419,0
Maryland,3/7/21,387319,3034546
Virginia,3/7/21,585700,0
Washington DC,3/6/21,41273,0
Maryland,3/6/21,386610,3032496
...,...,...,...
Virginia,1/31/20,0,0
Virginia,1/30/20,0,0
Virginia,1/29/20,0,0


In [78]:
# Narrow down data to date range 2020-03-01 to 2022-12-01
# Set date range for data
start_date = "2020-03-01"
end_date = "2022-12-01"
after_start_date = state_name_df["date"] >= start_date
before_end_date = state_name_df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates_1 = state_name_df.loc[between_two_dates]
filtered_dates_1

,date,positive,negative
state,,,


In [79]:
# Rename columns to match formatting of other data sets
state_name_df = state_name_df.rename(columns = {'positive': 'positive cases', 
                                                'negative': 'negative cases'})
state_name_df

,date,positive cases,negative cases
state,,,
Washington DC,3/7/21,41419,0
Maryland,3/7/21,387319,3034546
Virginia,3/7/21,585700,0
Washington DC,3/6/21,41273,0
Maryland,3/6/21,386610,3032496
...,...,...,...
Virginia,1/31/20,0,0
Virginia,1/30/20,0,0
Virginia,1/29/20,0,0


In [80]:
sorted_df = state_name_df.sort_values(["state", "date"])
sorted_df

,date,positive cases,negative cases
state,,,
Maryland,1/1/21,280219,2538142
Maryland,1/10/21,306674,2636065
Maryland,1/11/21,309686,2649578
Maryland,1/12/21,312351,2658742
Maryland,1/13/21,314867,2667562
...,...,...,...
Washington DC,9/5/20,14238,0
Washington DC,9/6/20,14279,0
Washington DC,9/7/20,14315,0


In [81]:
sorted_df.describe()

,positive cases,negative cases
count,1142.000000,1.142000e+03
mean,108292.510508,4.198975e+05
std,138005.047563,8.411837e+05
min,0.000000,0.000000e+00
25%,12326.250000,0.000000e+00
50%,44017.500000,0.000000e+00
75%,146894.500000,2.057062e+05
max,585700.000000,3.034546e+06


In [82]:
mean = sorted_df.groupby('state').mean()['positive cases']
median = sorted_df.groupby('state').median()['positive cases']
variance = sorted_df.groupby('state').var()['positive cases']
sds = sorted_df.groupby('state').std()['positive cases']
sems = sorted_df.groupby('state').sem()['positive cases']
summary_table_1 = pd.DataFrame({"Mean Positive Cases":mean,
                              "Median Positive Cases":median,
                              "Positive Cases Variance":variance,
                              "Positive Cases Std. Dev.":sds,
                              "Positive Cases Std. Err.":sems})
summary_table_1

,Mean Positive Cases,Median Positive Cases,Positive Cases Variance,Positive Cases Std. Dev.,Positive Cases Std. Err.
state,,,,,
Maryland,141397.706522,111219.0,1.368615e+10,116987.802929,6098.410795
Virginia,161540.955665,107054.0,2.962919e+10,172131.329906,8542.734466
Washington DC,16440.388587,14212.0,1.333106e+08,11546.019661,601.877881


In [83]:
sorted_df.to_csv('output_data/data_source_1.csv')

### Source 2 data clean up
Resource:  https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset?select=us-counties.csv
This is a US counties COVID-19 data set from Kaggle.com
This data is compiled from state and local governments and health departments
#### This will be output into output_data/data_source_2

In [84]:
covid_data_2 = "us-counties.csv"

# Read csv file into DataFrame, set state as index
covid_df_2 = pd.read_csv(covid_data_2, index_col = "state")
covid_df_2

,date,county,fips,cases,deaths
state,,,,,
Washington,1/21/2020,Snohomish,53061.0,1,0
Washington,1/22/2020,Snohomish,53061.0,1,0
Washington,1/23/2020,Snohomish,53061.0,1,0
Illinois,1/24/2020,Cook,17031.0,1,0
Washington,1/24/2020,Snohomish,53061.0,1,0
...,...,...,...,...,...
Wyoming,7/19/2020,Sweetwater,56037.0,173,2
Wyoming,7/19/2020,Teton,56039.0,208,1
Wyoming,7/19/2020,Uinta,56041.0,221,0


In [85]:
df_2 = covid_df_2[['date', 'cases']]

# Rename the 'cases' column to 'positive cases' to be consistent with other data sets
df_2 = df_2.rename(columns = {'cases': 'positive cases'})

states = ["District of Columbia", "Maryland", "Virginia"]
filter_df_2  = df_2[df_2.index.isin(states)]
filter_df_2

,date,positive cases
state,,
Maryland,3/5/2020,3
Maryland,3/6/2020,3
District of Columbia,3/7/2020,1
Maryland,3/7/2020,3
Virginia,3/7/2020,1
...,...,...
Virginia,7/19/2020,94
Virginia,7/19/2020,374
Virginia,7/19/2020,52


In [86]:
# Set date range for data
start_date = "2020-03-01"
end_date = "2022-12-01"
after_start_date = filter_df_2["date"] >= start_date
before_end_date = filter_df_2["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates_2 = filter_df_2.loc[between_two_dates]
filtered_dates_2

,date,positive cases
state,,


In [87]:
# Group duplicate dates for each state into a total positive case per day
group_df_2 = filter_df_2.groupby(["state", "date"])
group_df_2['positive cases'].sum()

state                 date     
District of Columbia  3/10/2020        4
                      3/11/2020       10
                      3/12/2020       10
                      3/13/2020       10
                      3/14/2020       16
                                   ...  
Virginia              7/5/2020     65748
                      7/6/2020     66102
                      7/7/2020     66740
                      7/8/2020     67375
                      7/9/2020     67988
Name: positive cases, Length: 407, dtype: int64

In [88]:
# Save data frame to a CSV file
group_df_2 = filter_df_2.groupby(["state", "date"])
group_df_2.sum().reset_index().to_csv('output_data/group_df_2.csv')

In [89]:
# Open newly saved data CSV file
new = "output_data/group_df_2.csv"
new_df_2 = pd.read_csv(new)
new_df_2

,Unnamed: 0,state,date,positive cases
0,0,District of Columbia,3/10/2020,4
1,1,District of Columbia,3/11/2020,10
2,2,District of Columbia,3/12/2020,10
3,3,District of Columbia,3/13/2020,10
4,4,District of Columbia,3/14/2020,16
...,...,...,...,...
402,402,Virginia,7/5/2020,65748
403,403,Virginia,7/6/2020,66102
404,404,Virginia,7/7/2020,66740
405,405,Virginia,7/8/2020,67375


In [90]:
new_df_2 = new_df_2[['state', 'date', 'positive cases']]

new_df_2['date'] = pd.to_datetime(new_df_2['date'])
data_source_2 = new_df_2.sort_values(["state", "date"])

# Set index to state
data_source_2 = data_source_2.set_index('state')

data_source_2

,date,positive cases
state,,
District of Columbia,2020-03-07,1
District of Columbia,2020-03-08,1
District of Columbia,2020-03-09,4
District of Columbia,2020-03-10,4
District of Columbia,2020-03-11,10
...,...,...
Virginia,2020-07-15,73527
Virginia,2020-07-16,74431
Virginia,2020-07-17,75433


In [91]:
# Source 2 summary
data_source_2.describe()

,positive cases
count,407.000000
mean,23798.390663
std,25404.241046
min,1.000000
25%,2712.000000
50%,10292.000000
75%,47336.500000
max,78698.000000


In [92]:
# summary table for source 2 data

mean = data_source_2.groupby('state').mean()['positive cases']
median = data_source_2.groupby('state').median()['positive cases']
variance = data_source_2.groupby('state').var()['positive cases']
sds = data_source_2.groupby('state').std()['positive cases']
sems = data_source_2.groupby('state').sem()['positive cases']
summary_table_2 = pd.DataFrame({"Mean Positive Cases":mean,
                              "Median Positive Cases":median,
                              "Positive Cases Variance":variance,
                              "Positive Cases Std. Dev.":sds,
                              "Positive Cases Std. Err.":sems})
summary_table_2

,Mean Positive Cases,Median Positive Cases,Positive Cases Variance,Positive Cases Std. Dev.,Positive Cases Std. Err.
state,,,,,
District of Columbia,5816.637037,6584.0,1.661169e+07,4075.743834,350.784178
Maryland,34883.467153,34174.0,7.648332e+08,27655.618251,2362.778923
Virginia,30530.844444,26746.0,6.654804e+08,25796.906708,2220.244224


In [93]:
# Save to CSV file
data_source_2.to_csv('output_data/data_source_2.csv')

### Source 3 data clean up
Resource:  https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
output into output_data/data_source_3


In [94]:

covid_url = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv"

# Read csv file into DataFrame, set state as index
covid_df_3 = pd.read_csv(covid_url, index_col = "state")
covid_df_3

,date,fips,cases,deaths
state,,,,
Washington,2020-01-21,53,1,0
Washington,2020-01-22,53,1,0
Washington,2020-01-23,53,1,0
Illinois,2020-01-24,17,1,0
Washington,2020-01-24,53,1,0
...,...,...,...,...
Virginia,2022-12-15,51,2173547,22709
Washington,2022-12-15,53,1872837,14929
West Virginia,2022-12-15,54,618893,7783


In [95]:
# Narrow down columns to state, date & cases
df_3 = covid_df_3[["date", "cases"]]
df_3

# Filter for DMV
states = ["District of Columbia", "Maryland", "Virginia"]
filter_df_3  = df_3[df_3.index.isin(states)]
# Rename column to match formatting in other data sets
filter_df_3 = filter_df_3.rename(columns = {'cases': 'positive cases'})
filter_df_3

,date,positive cases
state,,
Maryland,2020-03-05,3
Maryland,2020-03-06,3
District of Columbia,2020-03-07,1
Maryland,2020-03-07,3
Virginia,2020-03-07,1
...,...,...
Maryland,2022-12-14,1301829
Virginia,2022-12-14,2171414
District of Columbia,2022-12-15,172626


In [96]:
# Set timeframe parameters
start_date = '2020-03-01'
end_date = '2022-12-01'
after_start_date = filter_df_3["date"] >= start_date
before_end_date = filter_df_3["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates_3 = filter_df_3.loc[between_two_dates]
filtered_dates_3

,date,positive cases
state,,
Maryland,2020-03-05,3
Maryland,2020-03-06,3
District of Columbia,2020-03-07,1
Maryland,2020-03-07,3
Virginia,2020-03-07,1
...,...,...
Maryland,2022-11-30,1288582
Virginia,2022-11-30,2149512
District of Columbia,2022-12-01,171608


In [97]:
#Sort by date in ascending order
filtered_dates_3 = filtered_dates_3.sort_values(["state", "date"])
filtered_dates_3

,date,positive cases
state,,
District of Columbia,2020-03-07,1
District of Columbia,2020-03-08,1
District of Columbia,2020-03-09,4
District of Columbia,2020-03-10,4
District of Columbia,2020-03-11,10
...,...,...
Virginia,2022-11-27,2143804
Virginia,2022-11-28,2146026
Virginia,2022-11-29,2147680


In [98]:
# Source 3 summary
filtered_dates_3.describe()

,positive cases
count,3.002000e+03
mean,5.179550e+05
std,5.928599e+05
min,1.000000e+00
25%,5.696600e+04
50%,1.817910e+05
75%,8.659930e+05
max,2.151313e+06


In [99]:
mean = filtered_dates_3.groupby('state').mean()['positive cases']
median = filtered_dates_3.groupby('state').median()['positive cases']
variance = filtered_dates_3.groupby('state').var()['positive cases']
sds = filtered_dates_3.groupby('state').std()['positive cases']
sems = filtered_dates_3.groupby('state').sem()['positive cases']
summary_table_3 = pd.DataFrame({"Mean Positive Cases":mean,
                              "Median Positive Cases":median,
                              "Positive Cases Variance":variance,
                              "Positive Cases Std. Dev.":sds,
                              "Positive Cases Std. Err.":sems})
summary_table_3

,Mean Positive Cases,Median Positive Cases,Positive Cases Variance,Positive Cases Std. Dev.,Positive Cases Std. Err.
state,,,,,
District of Columbia,73130.048000,49810.0,3.463697e+09,58853.181263,1861.101003
Maryland,571368.702595,464483.5,1.781770e+11,422110.129676,13334.966027
Virginia,909259.426000,685845.5,5.196574e+11,720872.647061,22795.994676


In [100]:
# Save to csv file
filtered_dates_3.to_csv('output_data/data_source_3.csv')

## Population data clean

In [101]:
# Import population data

# Resource:  https://www.kaggle.com/headsortails/covid19-us-county-jhu-data-demographics/data?select=us_county.csv
# Saved as csv named population datasets

#Population data
#Load file to read
pop_file = ("us_county.csv")

# Read csv file into DataFrame
pop_df = pd.read_csv(pop_file, index_col = "state")
pop_df

,fips,county,state_code,male,female,median_age,population,female_percentage,lat,long
state,,,,,,,,,,
Alabama,1001,Autauga County,AL,26874,28326,37.8,55200,51.315217,32.534923,-86.642730
Alabama,1003,Baldwin County,AL,101188,106919,42.8,208107,51.376936,30.727479,-87.722564
Alabama,1005,Barbour County,AL,13697,12085,39.9,25782,46.873788,31.869581,-85.393210
Alabama,1007,Bibb County,AL,12152,10375,39.9,22527,46.055844,32.998628,-87.126475
Alabama,1009,Blount County,AL,28434,29211,40.8,57645,50.673953,33.980869,-86.567380
...,...,...,...,...,...,...,...,...,...,...
Puerto Rico,72145,Vega Baja Municipio,NaN,25580,27791,40.7,53371,52.071350,18.428461,-66.397926
Puerto Rico,72147,Vieques Municipio,NaN,4332,4439,43.6,8771,50.609965,18.122662,-65.439095
Puerto Rico,72149,Villalba Municipio,NaN,11169,11824,38.8,22993,51.424347,18.128155,-66.472816


In [102]:
# Narrow down columns of pop_df to state and population
narrow_pop_df = pop_df[["population", "lat", "long"]]
narrow_pop_df

,population,lat,long
state,,,
Alabama,55200,32.534923,-86.642730
Alabama,208107,30.727479,-87.722564
Alabama,25782,31.869581,-85.393210
Alabama,22527,32.998628,-87.126475
Alabama,57645,33.980869,-86.567380
...,...,...,...
Puerto Rico,53371,18.428461,-66.397926
Puerto Rico,8771,18.122662,-65.439095
Puerto Rico,22993,18.128155,-66.472816


In [103]:
# Filter out DMV states
states = ["District of Columbia", "Maryland", "Virginia"]
pop_filter_df  = narrow_pop_df[narrow_pop_df.index.isin(states)]
pop_filter_df

,population,lat,long
state,,,
District of Columbia,684498,38.904734,-77.016294
Maryland,71977,39.621450,-78.697617
Maryland,567696,39.006463,-76.605068
Maryland,827625,39.462711,-76.639292
Maryland,91082,38.543361,-76.568679
...,...,...,...
Virginia,89160,36.695318,-76.639839
Virginia,450135,36.733538,-76.043485
Virginia,21926,38.067411,-78.901356


In [104]:
state_pop_df = pop_filter_df.groupby('state')['population'].sum()
state_pop_df

# Save to csv file
state_pop_df.to_csv("output_data/state_pop_df.csv")

## The fifth data source contained information for state closings and reopening dates.
* Resource:  https://www.washingtonpost.com/graphics/2020/national/states-reopening-coronavirus-map/
*  This is a Washington Post website.  
* The data is gathered from state sites and from county and city sites for certain jurisdictions. 

#### This will be output into output_data/state_df